# To Do

include sources by paragraph:  
  https://python.langchain.com/docs/use_cases/question_answering/how_to/vector_db_qa  
  first split by paragraph   
  assign unique paragraph key  
  pass and prompt to gpt  

Allow for file upload:   
 https://www.gradio.app/docs/file  

implement streaming:  
  https://python.langchain.com/docs/modules/model_io/models/llms/streaming_llm  

improve gradio

works only with pdfs that have text layer

replies are not complete

# Import

In [24]:
from dotenv import load_dotenv
import os
import openai

# Langchain tools
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

# Gradio
import gradio as gr

# PDF tools (adding test layer)
import pytesseract
from pdf2image import convert_from_path

# Setup

In [35]:
# API key
load_dotenv()
openai.api_key  = os.environ['OPENAI_API_KEY']

# PDF path
pdf_path = "./pdf/Hochschild_2017_Teamplayer.pdf"
output_path = "./pdf/new_Hochschild_2017_Teamplayer.txt"

# Setup LLM
llm = OpenAI(temperature=0, top_p=0.1, n=1)

# Get text if there is text layer in pdf

In [ ]:
loader = PyPDFLoader(pdf_path)
docs = loader.load_and_split()

# Get text if there is no text layer in pdf

In [40]:
def extract_text_from_pdf(pdf_path):
    # Convert PDF to images
    images = convert_from_path(pdf_path)

    # Initialize OCR engine (Tesseract)
    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
    # Replace <path_to_tesseract> with the actual path to your Tesseract installation

    recognized_text = []

    # Perform OCR on each image
    for img in images:
        text = pytesseract.image_to_string(img, lang='eng')
        recognized_text.append(text)

    return recognized_text

# Example usage
# Replace 'path/to/input.pdf' with the actual PDF path
pdf_txt = extract_text_from_pdf(pdf_path)

# Print the recognized text for each page
for i, text in enumerate(docs):
    print(f"Page {i+1}:\n{text}\n")
    
def write_list_to_txt(lst, file_path):
    with open(file_path, 'w') as file:
        for item in lst:
            file.write(str(item) + '\n')

# Example list

# Specify the file path where you want to save the list
file_path = "./repo/temp.txt"

# Write the list to the text file
write_list_to_txt(pdf_txt, file_path)


Page 1:
Kapitel 10

Die Teamplayer: Loyalitit iiber alles

»Man sieht, dass ich Republikanerin bin«, sagt Janice Areno, als
sie mir in ihrem Biiro einen Platz anbietet. Elefanten, das Wappen-
tier der Republikaner, fiillen drei Regale an der Wand gegeniiber
von ihrem Schreibtisch. Einer ist aus blau-weifSem Porzellan, ein
zweiter golden, ein dritter rot, wei und blau und steht neben einer
Kinderzeichnung eines gelben Elefanten. Einer ist eine Teekanne
in Elefantenform. Ein weiterer halt eine amerikanische Flagge. Es
gibt grofe und kleine Elefanten, hdlzerne und gliserne, stehende
und gehende. Auer ihren Ehrungen fiir herausragende Dienste
an der Gemeinde und Fotos ihrer Verwandten hat sie diese Elefan-
ten im Laufe der Jahre bei Kuchenverkiufen, Tombolas und Par-
teitagen der Republikaner gesammelt. »Wenn ich einen Elefanten
sehe, empfinde ich Stolz auf dieses Land.«

Ich sitze Janice in ihrem geraumigen Biiro gegeniiber, in dem
sie seit Langem als Buchhalterin fiir die Grundstiickbewi

In [42]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader("./repo/temp.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1462, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 2151, which is longer than the specified 1000
Created a chunk of size 1101, which is longer than the specified 1000
Created a chunk of size 1112, which is longer than the specified 1000


# Embedding

In [43]:
embeddings = OpenAIEmbeddings()
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="Hochschild",
)

# Prompting

In [44]:
# Setup system prompt

prompt_template = """Use the following pieces of context to answer the question 
at the end. If you don't know the answer, just say that you don't know, 
don't try to make up an answer. Makes sure to answer the question thoroughly.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [45]:
# Give some user prompt examples

example_questions=[
    "What is the main argument of this paper?",
    "How does the author support the main argument?",
    "What is the author's conclusion?",
]

# Chat GUI

In [53]:
# Define QA function
# This works

def get_answers(query, history=[]):
    found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)
    chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)
    summary = chain.run(question = query, input_documents = found_docs)
    return summary

In [51]:
# Define QA function
# experimental

def get_answers_exp(query, history=[]):
    found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)
    chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)
    summary = chain.run(question = query, input_documents = found_docs)
    return summary

In [52]:
query = "was ist der hauptpunkt dieses textes?"
reply = get_answers_exp(query)
reply

"\nDer Hauptpunkt dieses Textes ist, wie Janice auf verschiedene soziale und ökologische Probleme reagiert. Sie glaubt, dass Kinder, die in Armut geboren werden, hart arbeiten und eine gute Bildung erhalten sollten, um aus dieser Situation herauszukommen. Sie sieht auch die Lösung darin, Kinder kirchlich einzubinden und die Fruchtbarkeit der armen Frauen einzuschränken. Sie spricht auch über die industrielle Umweltverschmutzung des Bayou d'Inde, wo ihr Onkel und seine Frau leben. Sie erkennt an, dass die Industrie notwendig ist, um Dinge wie Plastikflaschen, Gummischuhsohlen und Zahnpasta zu produzieren, aber sie ist traurig, dass das Land, das ihr Großvater bew"

In [48]:
# Run Gradio

pdf_chatter = gr.ChatInterface(fn=get_answers_exp, examples=example_questions, title="PDF Chatter")
# pdf_chatter.launch()

In [49]:
pdf_chatter.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
